In [84]:
import pandas as pd

In [85]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
sample_submission=pd.read_csv('sample_submission.csv')

In [86]:
train.shape, test.shape, sample_submission.shape

((7613, 5), (3263, 4), (3263, 2))

In [87]:
train.head()
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [88]:
train.drop(['keyword', 'location', 'id'], axis=1, inplace=True)
test.drop(['keyword', 'location', 'id'], axis=1, inplace=True)

In [89]:
print(train.head())
print(test.head())

                                                text  target
0  Our Deeds are the Reason of this #earthquake M...       1
1             Forest fire near La Ronge Sask. Canada       1
2  All residents asked to 'shelter in place' are ...       1
3  13,000 people receive #wildfires evacuation or...       1
4  Just got sent this photo from Ruby #Alaska as ...       1
                                                text
0                 Just happened a terrible car crash
1  Heard about #earthquake is different cities, s...
2  there is a forest fire at spot pond, geese are...
3           Apocalypse lighting. #Spokane #wildfires
4      Typhoon Soudelor kills 28 in China and Taiwan


In [90]:
training=train.values
training[:,0]
testing=test.values
testing

array([['Just happened a terrible car crash'],
       ['Heard about #earthquake is different cities, stay safe everyone.'],
       ['there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all'],
       ...,
       ['Green Line derailment in Chicago http://t.co/UtbXLcBIuY'],
       ['MEG issues Hazardous Weather Outlook (HWO) http://t.co/3X6RBQJHn3'],
       ['#CityofCalgary has activated its Municipal Emergency Plan. #yycstorm']],
      dtype=object)

In [91]:
x,y=training[:,0], training[:,1]

In [96]:
x[980]

'Louis Vuitton Monogram Sophie Limited Edition Clutch Cross body Bag - Full read by eBay http://t.co/VJgR6Liaxh http://t.co/55JR66PLOV'

In [97]:
from collections import Counter

In [98]:
z=Counter(y)
print("No. Of Real Disaster: ", z[1])
print("No. of Fake Disaster: ", z[0])
z

No. Of Real Disaster:  3271
No. of Fake Disaster:  4342


Counter({1: 3271, 0: 4342})

In [99]:
from sklearn.utils import shuffle
x,y=shuffle(x,y)

In [100]:
y=y.astype('float32')

In [101]:
y

array([0., 0., 0., ..., 1., 0., 1.], dtype=float32)

In [102]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
import re
from bs4 import BeautifulSoup

def tweets_to_words(tweet):
    nltk.download("stopwords", quiet=True)
    stemmer=PorterStemmer()
    
    text=BeautifulSoup(tweet, "html.parser").get_text()#remove html tag
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

In [103]:
tweets_to_words(x[210])

['stoni',
 'jackson',
 'america',
 'last',
 'hope',
 'lead',
 'armi',
 'felon',
 'thu',
 'armi',
 'reject',
 'armi',
 'satan',
 'http',
 'co',
 '0wbecdmhqo']

In [104]:
for i, tweet in enumerate(x):
    x[i]=tweets_to_words(tweet)

In [105]:
x[3]

['piprhi', 'predict', 'riot']

In [106]:
split_size=0.8
train_split=int(x.shape[0]*split_size)
x_train, y_train=x[:train_split], y[:train_split]
x,y=x[train_split:], y[train_split:]
val_split=int(0.5*x.shape[0])
x_val, y_val=x[:val_split], y[:val_split]
x_test, y_test=x[val_split:], y[val_split:]

print("X_train : ", x_train.shape)
print("Y_train : ", y_train.shape)
print("X_test : ", x_test.shape)
print("Y_test : ", y_test.shape)
print("X_val : ", x_val.shape)
print("Y_val : ", y_val.shape)

X_train :  (6090,)
Y_train :  (6090,)
X_test :  (762,)
Y_test :  (762,)
X_val :  (761,)
Y_val :  (761,)


In [70]:
import numpy as np

def build_dict(data, vocab_size=5000):
    words={}
    for tweet in data:
        for word in tweet:
            words[word]=words[word]+1 if word in words else 1
    #print(len(words))
    sorted_words=sorted(words, key=words.get, reverse=True)
    #print(len(sorted_words))
    #print(sorted_words)
    word_dict={}
    for i, word in enumerate(sorted_words[:vocab_size-2]):
        word_dict[word]=i+2
    #print(len(word_dict))
    return word_dict

In [71]:
word_dict = build_dict(x_train)

In [72]:
list(word_dict.items())[:20], len(word_dict)

([('co', 2),
  ('http', 3),
  ('like', 4),
  ('fire', 5),
  ('get', 6),
  ('bomb', 7),
  ('new', 8),
  ('via', 9),
  ('2', 10),
  ('one', 11),
  ('go', 12),
  ('peopl', 13),
  ('news', 14),
  ('kill', 15),
  ('burn', 16),
  ('year', 17),
  ('video', 18),
  ('crash', 19),
  ('time', 20),
  ('emerg', 21)],
 4998)

In [73]:
h=0
for tweet in x_train:
    if len(tweet)>h:
        h=len(tweet)
h

26

In [83]:
def convert_and_pad(word_dict, sentence, pad=26):
    noword=0
    infreq=1
    working_sentence=[noword] * pad
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index]=word_dict[word]
        else:
            working_sentence[word_index]=infreq
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=26):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [75]:
x_train, x_train_len=convert_and_pad_data(word_dict, x_train)
x_test, x_test_len=convert_and_pad_data(word_dict, x_test)
x_val, x_val_len=convert_and_pad_data(word_dict, x_val)

In [76]:
x_train[80]

array([   1,  186,  474,  510, 3400,  169, 3400, 2688,  588, 1904,  199,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [77]:
from torch.utils.data import TensorDataset, DataLoader
import torch
batch_size=256

features_train=torch.from_numpy(x_train)
targets_train=torch.from_numpy(y_train)
data = TensorDataset(features_train, targets_train)
train_loader=DataLoader(data, batch_size=batch_size)

features_val=torch.from_numpy(x_val)
targets_val=torch.from_numpy(y_val)
data = TensorDataset(features_val, targets_val)
val_loader=DataLoader(data, batch_size=batch_size)

features_test=torch.from_numpy(x_test)
targets_test=torch.from_numpy(y_test)
data = TensorDataset(features_test, targets_test)
test_loader=DataLoader(data, batch_size=batch_size)

In [78]:
import torch.nn as nn

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """
    def __init__(self, embedding_dim, hidden_dim, vocab_size, n_layers, dropout):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()
        self.hidden_dim=hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, batch_first=True)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = word_dict

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        #x = x.t()
        #lengths = x[0,:]
        #reviews = x[1:,:]
#         embeds = self.embedding(x)
#         lstm_out, _ = self.lstm(embeds)
#         out = self.dense(lstm_out)
#         out = out[lengths - 1, range(len(lengths))]
#         return self.sig(out.squeeze())
        batch_size=x.shape[0]
        x=x.long()
        #print(x.shape)
        x=self.embedding(x)
        #print(x.shape)
        x, _=self.lstm(x)
        #print(x.shape)
        x=x.contiguous().view(-1, self.hidden_dim)
        #print(x.shape)
        x=self.dense(x)
        x=x.view(batch_size, -1 , 1)
        #print(x.shape)
        x=self.sig(x)
        #print(x.shape)
        return x[:, -1]

In [79]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    clip=2
    valid_loss_min = np.Inf 
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        val_loss = 0
        for batch in train_loader:         
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            # TODO: Complete this train method to train the model provided.
            optimizer.zero_grad()
            out=model(batch_X)
            #print(out)
            #print(out)
            #nn.utils.clip_grad_norm_(model.parameters(), clip)
            loss=loss_fn(out.squeeze(), batch_y)
            
            #print(out.shape, batch_y.shape)
            loss.backward()
            optimizer.step()
            total_loss += loss.data.item()
        print("Epoch: {}, Training BCELoss: {}".format(epoch, total_loss / len(train_loader)))
        model.eval()
        for batch in val_loader:
            batch_x, batch_y=batch
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            out=model(batch_x)
            v_loss=loss_fn(out.squeeze(), batch_y)
            val_loss += v_loss.data.item()
        print(" Validation BCELoss: {}".format( val_loss / len(val_loader)))
        if val_loss<valid_loss_min:
            torch.save(model.state_dict(), 'model.pt')
            print("Saving..")
            valid_loss_min=val_loss

In [80]:
import torch.optim as optim
import torch 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(200, 128, 5000, 1, 0.3).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()


train(model, train_loader, 20, optimizer, loss_fn, device)

Epoch: 1, Training BCELoss: 0.6652032136917114
 Validation BCELoss: 0.6330903172492981
Saving..
Epoch: 2, Training BCELoss: 0.6089872544010481
 Validation BCELoss: 0.5995058417320251
Saving..
Epoch: 3, Training BCELoss: 0.5205310073991617
 Validation BCELoss: 0.5335490703582764
Saving..
Epoch: 4, Training BCELoss: 0.4144553989171982
 Validation BCELoss: 0.5323665340741476
Saving..
Epoch: 5, Training BCELoss: 0.34224448539316654
 Validation BCELoss: 0.5420705676078796
Epoch: 6, Training BCELoss: 0.3009832625587781
 Validation BCELoss: 0.5209340552488962
Saving..
Epoch: 7, Training BCELoss: 0.2516945277651151
 Validation BCELoss: 0.5830248792966207
Epoch: 8, Training BCELoss: 0.1968051651492715
 Validation BCELoss: 0.6513025164604187
Epoch: 9, Training BCELoss: 0.1732218541825811
 Validation BCELoss: 0.6640480558077494
Epoch: 10, Training BCELoss: 0.15703397647788128
 Validation BCELoss: 0.7135430375734965
Epoch: 11, Training BCELoss: 0.17773895027736822
 Validation BCELoss: 0.7740832169

In [81]:
total_acc_n=0
total_acc_d=0
test_loss=0
for batch in test_loader:
    model.eval()
    batch_x,batch_y=batch
    batch_x = batch_x.to(device)
    batch_y = batch_y.to(device)
    out=model(batch_x)
    loss=loss_fn(out.squeeze(), batch_y)
    test_loss+=loss.data.item()
    #print(out.squeeze().shape, batch_y.shape)
    acc=[1 if i == True else 0 for i  in (torch.round(out.squeeze()) == batch_y)  ]
    #print(acc)
    total_acc_n+=sum(acc)
    total_acc_d+=len(acc)

In [82]:
total_acc_n/total_acc_d, test_loss/len(test_loader)

(0.7742782152230971, 0.9497159520785013)

In [198]:
test=pd.read_csv('test.csv')
sample_submission=pd.read_csv('sample_submission.csv')

In [199]:
sample_submission.head(), test.head()

(   id  target
 0   0       0
 1   2       0
 2   3       0
 3   9       0
 4  11       0,
    id keyword location                                               text
 0   0     NaN      NaN                 Just happened a terrible car crash
 1   2     NaN      NaN  Heard about #earthquake is different cities, s...
 2   3     NaN      NaN  there is a forest fire at spot pond, geese are...
 3   9     NaN      NaN           Apocalypse lighting. #Spokane #wildfires
 4  11     NaN      NaN      Typhoon Soudelor kills 28 in China and Taiwan)

In [200]:
test.drop(['keyword', 'location'], axis=1, inplace=True)

In [201]:
test.head()

,id,text
0,0,Just happened a terrible car crash
1,2,"Heard about #earthquake is different cities, s..."
2,3,"there is a forest fire at spot pond, geese are..."
3,9,Apocalypse lighting. #Spokane #wildfires
4,11,Typhoon Soudelor kills 28 in China and Taiwan


In [202]:
submission_test=test['text'].values

In [203]:
submission_test

array(['Just happened a terrible car crash',
       'Heard about #earthquake is different cities, stay safe everyone.',
       'there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all',
       ..., 'Green Line derailment in Chicago http://t.co/UtbXLcBIuY',
       'MEG issues Hazardous Weather Outlook (HWO) http://t.co/3X6RBQJHn3',
       '#CityofCalgary has activated its Municipal Emergency Plan. #yycstorm'],
      dtype=object)

In [204]:
for i, tweet in enumerate(submission_test):
    submission_test[i]=tweets_to_words(tweet)

In [205]:
submission_test, _ =convert_and_pad_data(word_dict, submission_test)

In [206]:
submission_test[0]

array([ 220, 1953,   54,   20,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0])

In [207]:
submission_test_tensor=torch.from_numpy(submission_test).cuda()
out=model(submission_test_tensor)
out=torch.round(out)

In [208]:
out=out.squeeze()

In [209]:
out=out.cpu().detach().numpy()

In [210]:
out=out.astype(int)

In [211]:
test.head()

,id,text
0,0,"[happen, terribl, car, crash]"
1,2,"[heard, earthquak, differ, citi, stay, safe, e..."
2,3,"[forest, fire, spot, pond, gees, flee, across,..."
3,9,"[apocalyps, light, spokan, wildfir]"
4,11,"[typhoon, soudelor, kill, 28, china, taiwan]"


In [212]:
test.drop('text', axis=1, inplace=True)
test['target']=out

In [217]:
test.to_csv('final_submission.csv', index=False)

In [214]:
test

,id,target
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,0
3261,10874,1


In [215]:
test['target'].dtype

dtype('int32')